### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# sample num
sending_sample_num = 400  # 150,200,300,500

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = English
Lang Id = 1
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/3-Word Group In Youtube Sentence Firebase"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
##cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
#app = firebase_admin.initialize_app(cred)

In [7]:
#fire_db = firestore.client()

#### Read Data

In [8]:
#parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Result/2-Word Group Prepare For Youtube Tool"

In [9]:
#df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28 = df_28.head(sending_sample_num)
df_28

,video_id,start_time,end_time,text
0,mO0oqUQ5eHc,2037,2040,and this is what it was for me and it was
1,7eUrrQRpz2w,2007,2010,what do we have to do and not do it this
2,UNnFRj9_TNo,2226,2228,and what this will do is this will
3,fgMD2wvpvpk,274,279,and what this will do is this will
4,Gyqu5AER7gg,624,626,what it is that you do and do not
...,...,...,...,...
286,yeLQfMmbbcE,60,62,no i am
287,uldlsT98Bbs,289,293,it will
288,LfnrRPFhkuY,639,643,it will
289,zxiHkW22UrA,314,317,it will


In [10]:
#df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200 = df_200.head(sending_sample_num)
df_200

,video_id,start_time,end_time,text
0,_aBzOI9DWb4,538,546,what do you think about this i mean yeah to me...
1,ldlflaQVAAE,937,943,things that are out there you can do nothing a...
2,MXGmKbjEIlw,2610,2615,the way is where this is why i really want to ...
3,dYa5scWadpc,1637,1645,say oh yeah yes she can and she would like to ...
4,fhADSlQLimI,100,109,help give back how can i make it that much bet...
...,...,...,...,...
395,0F4N_GXNBg8,6286,6289,there was some money for some but just
396,9dJjpw42ZPw,8,12,he was not doing things like he should
397,OWrKn2raz_c,2105,2109,okay all right but yeah here are other
398,sE91KaP0RLk,110,113,feel like home never before in my life


In [11]:
#df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000 = df_1000.head(sending_sample_num)
df_1000

,video_id,start_time,end_time,text
0,K1GhQ8633Bg,293,302,so you can imagine if you go through all of th...
1,G3w3f6M_w54,971,978,and so one of the questions people say to me i...
2,GymR27z8RHM,49,59,its a little family that you turn into and jus...
3,J6eEdQrcYZw,2153,2161,than you would ever imagine right and its its ...
4,ka7lOSmTe68,578,583,and so those are some of the things that we go...
...,...,...,...,...
395,Tjn7qSFT_oM,25,33,my love he makes me feel like nobody else nobo...
396,QboXCnKpgtU,350,353,but nobody would wan na tell me what those thi...
397,SzY1NQM8N0I,3060,3065,if other people have just last questions ask t...
398,MbLu8yDx500,117,122,because those things can feel like too much so...


In [12]:
#df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000 = df_5000.head(sending_sample_num)
df_5000

,video_id,start_time,end_time,text
0,iJkfTPN8MS0,2093,2110,without having um as much structure or as much...
1,nDOjm1UKsNM,1869,1886,lets talk about your action plans now and i an...
2,nDOjm1UKsNM,932,951,here in as an example of hours of operation ri...
3,b9pudZegYgQ,2320,2335,norman jones remember the kind of big picture ...
4,pNDXsUrO3Rs,2798,2809,de rose we appreciate you being here im glad i...
...,...,...,...,...
395,cOOdxOtpj5U,182,187,get therapy or any any further support its won...
396,3O8UGcZQ1dU,175,180,symptoms can not then be explained by drugs al...
397,gghRXOXY820,742,747,she can only use her special power once like m...
398,P6wsvgqXz_A,78,83,my left hand with me across here because its k...


In [13]:
#df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000 = df_10000.head(sending_sample_num)
df_10000

,video_id,start_time,end_time,text
0,rZEc_NCcorU,2165,2183,dr michelle taylor good afternoon everyone its...
1,b9pudZegYgQ,2322,2339,norman jones remember the kind of big picture ...
2,pNDXsUrO3Rs,2799,2810,de rose we appreciate you being here im glad i...
3,6dxtMxcFJ3M,3050,3067,when all the students parents old school toget...
4,o8vvI_sbAW0,2825,2842,eric fletcher but an overall the students are ...
...,...,...,...,...
395,jZlbmdYB4T8,207,213,production plant can produce different product...
396,4g_8_KK8j_g,584,591,be smaller than your actual email list because...
397,hVbOMHu1yrg,448,454,college students considered online learning mo...
398,qGF6aqjMcDQ,660,664,forget solving test chambers just think how mu...


In [14]:
#df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000 = df_20000.head(sending_sample_num)
df_20000

,video_id,start_time,end_time,text
0,gasLdUX61TU,5512,5527,that this is not just getting any old attorney...
1,HWTd4jxCixk,2198,2213,thats something you dont have to worry about a...
2,qgBqBnEJFgs,1443,1462,so prepare relevant questions and also on the ...
3,9nBekn1eQtg,3463,3484,james moore depending on the graduation someti...
4,Anatl2PLRag,3705,3724,this type of research yeah i think thats still...
...,...,...,...,...
395,dvA3CXJISu4,2077,2082,who could help them become more strategic abou...
396,VaxNN3YRhBA,2047,2054,understood because its training setup was look...
397,Iu3y8E0AwAE,20,25,but they could also be used on regular flip ma...
398,O9QrSV1UXbc,13,25,be it businessman job seeker students housewif...


In [15]:
#df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = pd.read_excel(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000 = df_40000.head(sending_sample_num)
df_40000

,video_id,start_time,end_time,text
0,gasLdUX61TU,1756,1772,brian hoffman but look at how different the ou...
1,SnAFp1hTNP8,5444,5464,chrissy foster were seeing this pop up in so m...
2,twJkEtTvvEo,485,503,and that is something thats really important e...
3,PRG60EnKnIo,2541,2554,david shoemaker responded back to her in a pri...
4,tffd1kmM3e4,2082,2102,teresa bill well and ill just share that i rea...
...,...,...,...,...
395,1pHrVACiVaM,1557,1564,deliver very high margin simple services thats...
396,hg3YADOxD1w,1765,1773,using cdc social vulnerability index about peo...
397,vImD3IValNk,987,993,high temperatures stagnant air storage spaces ...
398,Rl1WWhHrobk,440,445,while were talking about power hungry applianc...


##### Calculate Speech Time Mean

In [16]:
(df_28["end_time"] - df_28["start_time"]).mean()

2.59106529209622

In [17]:
(df_200["end_time"] - df_200["start_time"]).mean()

4.45

In [18]:
(df_1000["end_time"] - df_1000["start_time"]).mean()

5.5975

In [19]:
(df_5000["end_time"] - df_5000["start_time"]).mean()

7.7075

In [20]:
(df_10000["end_time"] - df_10000["start_time"]).mean()

8.0

In [21]:
(df_20000["end_time"] - df_20000["start_time"]).mean()

8.3825

In [22]:
(df_40000["end_time"] - df_40000["start_time"]).mean()

9.22

#### Sending Data To Firestore

In [23]:
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [24]:
fire_db = firestore.client()

##### Language Data

In [25]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [26]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [27]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [28]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [29]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [30]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [31]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [32]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [33]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [34]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [35]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [36]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [37]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [38]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [39]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [40]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [41]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [42]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [43]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [44]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [45]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [46]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [47]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [48]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [49]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [50]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [51]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [52]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [53]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [54]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [55]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [56]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [57]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1683640715
  nanos: 263676000
}